# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "output_data/cities.csv"

cities_df = pd.read_csv(file)

cities_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Atuona,PF,-9.80,-139.03,81.46,73,13,14.43,1588814312
1,Punta Arenas,CL,-53.15,-70.92,41.00,93,75,2.24,1588814426
2,Kavaratti,IN,10.57,72.64,85.59,71,95,4.65,1588814345
3,Kaitangata,NZ,-46.28,169.85,59.59,62,42,13.82,1588814427
4,Thompson,CA,55.74,-97.86,33.80,37,5,11.41,1588814427
...,...,...,...,...,...,...,...,...,...
563,Bathsheba,BB,13.22,-59.52,80.60,78,40,20.80,1588814566
564,Tiznit Province,MA,29.58,-9.50,63.45,79,100,2.55,1588814567
565,Ipatovo,RU,45.72,42.90,56.32,92,100,12.50,1588814567
566,Adeje,ES,28.12,-16.73,70.00,72,40,10.29,1588814567


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
humidity = cities_df['Humidity'].astype(float)
locations = cities_df[['Lat','Lng']]

In [4]:
fig = gmaps.figure(center = [2,5], zoom_level =1.9)

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, dissipating = False, point_radius=3)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create a temperature range suitable for me from original DataFrame
temp_df = cities_df[(cities_df['Max Temp'] >= 65) & (cities_df['Max Temp'] >=85)]

# Create a maximum wind range suitable for me from original DataFrame
temp_df = temp_df[temp_df['Wind Speed'] <= 15]

# Create a threshold of clouds suitable for me from original DataFrame
temp_df = temp_df[temp_df['Cloudiness'] <= 10]

# Clear out any not a number values
temp_df = temp_df.dropna(how='any')


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Copy my optimun settings for vacation destination to required DataFrame
hotel_df = temp_df

#Insert new column for hotel name
hotel_df['Hotel Name'] = ""

#hotel_df

In [10]:
# set params for hotel search
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
   
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    params["location"] = f"{lat},{lng}"
    
  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
   
    hotel_name = requests.get(base_url, params=params)
    
    
    hotel_name = hotel_name.json()
    
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("No Hotels Found, Skipping...")

hotel_df


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 31.9429495,
                    "lng": -112.8111855
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.94430117989272,
                        "lng": -112.8098359701073
                    },
                    "southwest": {
                        "lat": 31.94160152010728,
                        "lng": -112.8125356298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/camping-71.png",
            "id": "511c5ab32e51da804024d273b778474ed6720069",
            "name": "Twin Peaks Campground",
            "photos": [
                {
                    "height": 1200,
                    "html_attributions": [
                        "<a href=\"https://maps.

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAP3lq9SGocRw0CCal4Ra9HAty6OrvWzULo3bRD4gXJZjzHKH0rJHSTAo52Ab-4UIZJWggPxM2RWybgCjUytPQupTGz85oI4ZS4kSvUQsqBujn1twDIol5nh48tk8cFbHgPFXN-H8Qwmy-Ex_wIIKABiwe9WSRVBM-1YccSBtHY7DEZ2q6IJT3rtiRaf5MDJIFSqRtN4DwPeV9bLcflvJL4okON-FoACTAkQiN6ffLn_xsAXCFC2vOfSliJCkg0Z7fPocvRk_if5qGOPptKIo10-0t-S-hS_q4FtAuK0AqqiBZqHyq5G35nEA7DCsUavDYA3dLfGSu3cIrkUX3vRH552tp1lQQQnmEk2WgeLTzldwQZ4pGV4jM4gMwZtYoVlQgCqS716pL1PrFrqs7uNRXBwSEPJ1WUpnktcWEw1k2NrByikaFL658JSl2XYfwxMM5AlNuaj8Qytv",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.1676438,
                    "lng": -113.0142294
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.16894122989272,
                        "lng": -113.0124542201073
                    },
                    "southwest": {
                        

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.4908504,
                    "lng": -13.0629797
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.49198337989272,
                        "lng": -13.06119037010728
                    },
                    "southwest": {
                        "lat": 20.48928372010728,
                        "lng": -13.06389002989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "88818bb171dd6f51f39d8f90910660b957ea5ba1",
            "name": "Odar kanawal",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3024,
                    "h

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
No Hotels Found, Skipping...
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
No Hotels Found, Skipping...
{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAIjzrv5NSIXTTeD8bqRsIkghUa_5A1pUqpuM6EtX8u1GUYDVnvQNWUdTOILOxZ5_fZ-L1kR3NDddo4CVFs-UmokmAVBd0g5krrtJWQ5XNwIzNrm_xrBUK39e6x1IdwhvaOD0UFORS-6NIv2rbz0tCoHvw4_WgnPp7tZdhpM1CWbUJBnuxgmzXW8YjUrC5j2id1majzzmGnLQglkQ7KEOeDEReuiNlxlH43L4u1MX_eSK4NeAf9byitiL7m6jMze2DsR9sEOHb9mrNIMZI_S2TzN7okc-Lh2l5UaX-DDtAc_PbBZtwuMwR5mWWQmtE8URIloAwyJhZ56JqNffxiBL3Fi8fOweLrnF4hM-paQLogYS898cEGzSIPNSV9towUjKLVMvmJvQWbaxwfa2TOlt0xESEH_MNGnpc60cnQOO29IZo1QaFDvN6gHuCL9X16O6cwE45CVxdcfW",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 23.00682,
                    "lng": -109.714283
                },
                "viewport": {


{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAMQo0ahHoNexlB9ftnicQYt4LZG0kx7lrklpkpvFHoOjnlqxCPSleITusYRXJScqWmGiBvpjGfuaSnDDaJytnIMpHrFfYikNKSrA0FwStiwD2HaTAc40skYp8qOykvgyOjmtSbgUnOuMF9RqM0Jj8d76og_UFajGtXsGcu9Avur0w0WQHvbF8G4aN2R52XOHBL2vMoODDsKztNvATh3_IQiBKKTv0TnamMyJYoyFOdsYgkU43eyjWJBXwIKu6Yh6BzgY2XyUAvPRNigCkw1kuk6eKLXiCMqfp4gkKaewfz7dbz8U3M6jkELb0EcgKruXL67YPUFfhBE_MUepjX3cLDcyWmZ3KW4lkemaSpAr2ySl7lVsQPwB4dR-89Gwc3Q_KYfIWojouOamdgEGqxaSly0SEKnV8H8t_NbpxU7pPZwT0ioaFJwflgtV0AEQHr-39ZYrAdLPqn7A",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 21.2835384,
                    "lng": -157.8313227
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.28475157989272,
                        "lng": -157.8298504201072
                    },
                    "southwest": {
                        

...

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
No Hotels Found, Skipping...
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 10.279142,
                    "lng": 11.1730615
                },
                "viewport": {
                    "northeast": {
                        "lat": 10.28049272989272,
                        "lng": 11.17440302989272
                    },
                    "southwest": {
                        "lat": 10.27779307010728,
                        "lng": 11.17170337010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f5e3bb66211924d466f777ff7161fef9a9b5a44c",
            "name": "LELAH VISITORS SUITE",
            "opening_hours": {
                "open_now": true
 

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAMfkYxx51W_Dr1JoMY51eNwD8DY6_D6qZjSy8cYOvR34udiUvj8f_oS7bHxsrzMewVJ15UL1X9jnyw5dSeQeghajHCO6fn8wVxuzjj80OdodVIpQwUCwl7YR3_ed_xmhtFZRrY57_SpU2dKnM44yOTE_O2vMomvpm52suCmEKB6R0H9fVlXuf4cVWL4axFLjRE3XZMbkmMYOTPQrGaxMITuNKb1fhETY4RzxBJXWNJpBeKgVBVgKKGRFTRMuvbatl90ke49WJPNv5JSs2vulDSWV6zM_taH2fPLfk0GRW5n1e973WHLLWhbUTb42tvfCch-kM_o0W9ljWEW6ochjd9_jJiz9vGlZtKe4mgWh8YuiGKkMpEAgNW3Jn9pgoSUHzjldU53kyLlferN69lAKaWASEG0gNWIJB6SXg66nk9DypIoaFBJUcdkzfonodQcXMzki8Iydi7Ez",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 12.0889297,
                    "lng": 15.0712221
                },
                "viewport": {
                    "northeast": {
                        "lat": 12.09029312989272,
                        "lng": 15.07256897989272
                    },
                    "southwest": {
                        "la

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -0.9879494,
                    "lng": 122.7860761
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.9867217201072778,
                        "lng": 122.7870448298927
                    },
                    "southwest": {
                        "lat": -0.9894213798927219,
                        "lng": 122.7843451701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "83d8aeceeb2fe4ea238ca1f8db798a2bc1fc8a19",
            "name": "Swiss-Belinn Hotel Luwuk",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1385,
        

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.8837377,
                    "lng": 5.2981269
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.88531147989272,
                        "lng": 5.299487929892722
                    },
                    "southwest": {
                        "lat": 14.88261182010728,
                        "lng": 5.296788270107278
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "a2e3ee78f69d44097fb474aab80e93e9517099e6",
            "name": "HOTEL TARKA",
            "photos": [
                {
                    "height": 2160,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
17,Sonoita,MX,31.85,-112.83,100.99,5,0,8.99,1588814430,Twin Peaks Campground
99,Socorro,US,31.65,-106.30,93.20,9,5,5.82,1588814451,Holiday Inn El Paso Airport
110,Saint George,US,37.10,-113.58,91.00,7,1,8.05,1588814453,Majestic View Lodge
121,Atar,MR,20.52,-13.05,85.32,14,0,7.99,1588814456,Odar kanawal
202,Adrar,MR,20.50,-10.07,88.99,12,0,5.97,1588814476,
223,Umm Hājar,TD,13.30,19.70,85.05,39,1,9.48,1588814481,
264,Cabo San Lucas,MX,22.89,-109.91,86.00,12,5,11.41,1588814491,One&Only Palmilla
302,Makakilo City,US,21.35,-158.09,86.00,42,1,8.97,1588814501,Hotel LaCroix
303,Bandiagara,ML,14.35,-3.61,95.00,26,2,6.93,1588814501,Hotel de la Falaise
363,Todos Santos,MX,23.45,-110.22,90.00,58,5,10.29,1588814515,Cerritos Beach Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))